<a href="https://colab.research.google.com/github/kelecheeii/ML-Code/blob/main/Insurance_Fraud_Detection_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 6.8 MB/s eta 0:00:00


In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00


In [3]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import warnings

warnings.filterwarnings('ignore')

In [5]:
from google.colab import files

uploaded = files.upload()

Saving fraud_oracle.csv to fraud_oracle.csv


In [6]:
data = pd.read_csv('fraud_oracle.csv')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Month                 15420 non-null  object
 1   WeekOfMonth           15420 non-null  int64 
 2   DayOfWeek             15420 non-null  object
 3   Make                  15420 non-null  object
 4   AccidentArea          15420 non-null  object
 5   DayOfWeekClaimed      15420 non-null  object
 6   MonthClaimed          15420 non-null  object
 7   WeekOfMonthClaimed    15420 non-null  int64 
 8   Sex                   15420 non-null  object
 9   MaritalStatus         15420 non-null  object
 10  Age                   15420 non-null  int64 
 11  Fault                 15420 non-null  object
 12  PolicyType            15420 non-null  object
 13  VehicleCategory       15420 non-null  object
 14  VehiclePrice          15420 non-null  object
 15  FraudFound_P          15420 non-null

In [9]:
# scaling and encoding algorithms
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder

# column transformer
from sklearn.compose import ColumnTransformer, make_column_selector

# handling outliers
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# variance threshold and RFECV
from sklearn.feature_selection import VarianceThreshold, RFECV
from sklearn.model_selection import StratifiedKFold

# Machine learning algorithms
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

# SMOTE oversampling
from imblearn.over_sampling import SMOTE

# Machine Learning - Optuna
import optuna
from optuna.samplers import TPESampler

# Train test split
from sklearn.model_selection import train_test_split

# Pipelines
from imblearn.pipeline import Pipeline

# Machine learning - feature importance

In [10]:
# checking raw data for missing values

data.isnull().sum()

,0
Month,0
WeekOfMonth,0
DayOfWeek,0
Make,0
AccidentArea,0
DayOfWeekClaimed,0
MonthClaimed,0
WeekOfMonthClaimed,0
Sex,0
MaritalStatus,0


In [11]:
# Creating a summary DataFrame to understand the dataset
summary = pd.DataFrame({
    'Column Name': data.columns,
    'Data Type': data.dtypes.values,
    'Unique Values Count': [data[col].nunique() for col in data.columns]
})

# Displaying the summary of the dataset
print(summary)

             Column Name Data Type  Unique Values Count
0                  Month    object                   12
1            WeekOfMonth     int64                    5
2              DayOfWeek    object                    7
3                   Make    object                   19
4           AccidentArea    object                    2
5       DayOfWeekClaimed    object                    8
6           MonthClaimed    object                   13
7     WeekOfMonthClaimed     int64                    5
8                    Sex    object                    2
9          MaritalStatus    object                    4
10                   Age     int64                   66
11                 Fault    object                    2
12            PolicyType    object                    9
13       VehicleCategory    object                    3
14          VehiclePrice    object                    6
15          FraudFound_P     int64                    2
16          PolicyNumber     int64              

In [12]:
copy_d = data.copy()

In [13]:
# Identifying boolean-like columns in the dataset
# A boolean-like column is defined as one with only 2 unique values.
bool_val = [col for col in copy_d.columns if copy_d[col].nunique() <= 2]

# Display the values in those columns
copy_d[bool_val].head()

,AccidentArea,Sex,Fault,FraudFound_P,PoliceReportFiled,WitnessPresent,AgentType
0,Urban,Female,Policy Holder,0,No,No,External
1,Urban,Male,Policy Holder,0,Yes,No,External
2,Urban,Male,Policy Holder,0,No,No,External
3,Rural,Male,Third Party,0,Yes,No,External
4,Urban,Female,Third Party,0,No,No,External


In [14]:
# Define the target column
target_col = 'FraudFound_P'

# Initialize a dictionary to store label mappings for each column
label_mappings = {}

# Apply LabelEncoder to each column in bool_val
for col in bool_val:
    if col != target_col:  # Exclude the target column
        label_encoder = LabelEncoder()
        copy_d[col] = label_encoder.fit_transform(copy_d[col])
        # Store the label mappings for this column
        label_mappings[col] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Print the label mappings for all columns
for col, mapping in label_mappings.items():
    print(f"Column: {col}, Mapping: {mapping}")

Column: AccidentArea, Mapping: {'Rural': 0, 'Urban': 1}
Column: Sex, Mapping: {'Female': 0, 'Male': 1}
Column: Fault, Mapping: {'Policy Holder': 0, 'Third Party': 1}
Column: PoliceReportFiled, Mapping: {'No': 0, 'Yes': 1}
Column: WitnessPresent, Mapping: {'No': 0, 'Yes': 1}
Column: AgentType, Mapping: {'External': 0, 'Internal': 1}


In [15]:
# Defining columns that are deemed unnecessary for the analysis
useless_columns = ['Month', 'WeekOfMonth', 'DayOfWeek', 'DayOfWeekClaimed', 'WeekOfMonthClaimed', 'PolicyNumber']

# Dropping the identified unnecessary columns from the dataset
copy_d = copy_d.drop(useless_columns, axis=1)

# Displaying the first few rows of the modified dataset
copy_d.head()

,Make,AccidentArea,MonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Honda,1,Jan,0,Single,21,0,Sport - Liability,Sport,more than 69000,...,3 years,26 to 30,0,0,0,none,1 year,3 to 4,1994,Liability
1,Honda,1,Jan,1,Single,34,0,Sport - Collision,Sport,more than 69000,...,6 years,31 to 35,1,0,0,none,no change,1 vehicle,1994,Collision
2,Honda,1,Nov,1,Married,47,0,Sport - Collision,Sport,more than 69000,...,7 years,41 to 50,0,0,0,none,no change,1 vehicle,1994,Collision
3,Toyota,0,Jul,1,Married,65,1,Sedan - Liability,Sport,20000 to 29000,...,more than 7,51 to 65,1,0,0,more than 5,no change,1 vehicle,1994,Liability
4,Honda,1,Feb,0,Single,27,1,Sport - Collision,Sport,more than 69000,...,5 years,31 to 35,0,0,0,none,no change,1 vehicle,1994,Collision


In [16]:
# Convert 'RepNumber' column from int to str
copy_d['RepNumber'] = copy_d['RepNumber'].astype(str)

In [17]:
copy_d.dtypes

,0
Make,object
AccidentArea,int64
MonthClaimed,object
Sex,int64
MaritalStatus,object
Age,int64
Fault,int64
PolicyType,object
VehicleCategory,object
VehiclePrice,object


In [18]:
# Separating the dataset into features (X) and target variable (y)
X = copy_d.drop('FraudFound_P', axis=1)
y = copy_d['FraudFound_P']

In [19]:
# Splitting the dataset into training and testing sets
# Allocating 25% of the data for testing, Setting a random seed for reproducibility.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=47)

In [21]:
lightgbm_model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=42, verbose=-1)

In [23]:
# Define column groups

numeric_columns = ['Age', 'Deductible', 'Year']
ordinal_columns = ['BasePolicy']
nominal_columns = ['VehiclePrice', 'AgeOfVehicle', 'Make', 'MonthClaimed', 'MaritalStatus',
                   'PolicyType', 'VehicleCategory', 'Days_Policy_Accident',
                   'Days_Policy_Claim', 'PastNumberOfClaims', 'AgeOfPolicyHolder', 'NumberOfSuppliments',
                   'AddressChange_Claim', 'NumberOfCars', 'RepNumber']


preprocessor = ColumnTransformer(
    transformers=[
        ('ordinal', OrdinalEncoder(), ordinal_columns),
        ('nominal', OneHotEncoder(handle_unknown='ignore', sparse_output=False), nominal_columns),
        ('numeric', StandardScaler(), numeric_columns)])

# Preprocess the data before applying SMOTE
X_train_preprocessed = preprocessor.fit_transform(X_train)

# Apply SMOTE separately before fitting the pipeline
smote = SMOTE(random_state=47)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_preprocessed, y_train)

# Feature selectiion using RFECV
rfecv = RFECV(estimator=lightgbm_model, step=1, cv=3, scoring='roc_auc', n_jobs=-1)
rfecv.fit(X_train_resampled, y_train_resampled)



RFECV(cv=3, estimator=LGBMClassifier(random_state=42, verbose=-1), n_jobs=-1,
      scoring='roc_auc')

In [26]:
X_selected = rfecv.transform(X_train_resampled)

onehot_feature_names = preprocessor.named_transformers_['nominal'].get_feature_names_out(nominal_columns)

# Combine all feature names
all_feature_names = np.concatenate([ordinal_columns, onehot_feature_names, numeric_columns])

# Get selected feature names
selected_feature_names = all_feature_names[rfecv.support_]

# Create DataFrame with selected features and names
X_final = pd.DataFrame(X_selected, columns=selected_feature_names) # Convert to dense array if needed

In [27]:

# Define the objective function for Optuna
def objective(trial):
    # Suggest classifier type
    classifier_name = trial.suggest_categorical(
        'classifier', ['RandomForest', 'GradientBoosting', 'BalRandomForest', 'SVC', 'LGBMClassifier']
        )

    # Hyperparameters for RandomForest
    if classifier_name == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 100, 500)
        max_depth = trial.suggest_int('max_depth', 1, 3)
        classifier = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=47
        )

    # Hyperparameters for LGBMClassifier
    elif classifier_name == 'LGBMClassifier':
        n_estimators = trial.suggest_int('n_estimators', 100, 500)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 1.0)
        num_leaves = trial.suggest_int('num_leaves', 2, 15)
        classifier = LGBMClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            num_leaves=num_leaves,
            random_state=47
        )

     # Hyperparameters for GradientBoosting
    elif classifier_name == 'GradientBoosting':
        n_estimators = trial.suggest_int('n_estimators', 100, 400)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 2.0)
        max_depth = trial.suggest_int('max_depth', 10, 20)
        classifier = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            random_state=47
        )

    # Hyperparameters for SVC
    elif classifier_name == 'SVC':
        C = trial.suggest_float('C', 0.01, 1.0)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf'])
        classifier = SVC(
            kernel=kernel,
            C=C,
            random_state=47,
            probability=True
        )


    # Hyperparameters for BalRandomForest
    else:  # classifier_name == 'BalRandomForest'
        n_estimators = trial.suggest_int('n_estimators', 100, 500)
        class_weight = trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample'])
        max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
        max_depth = trial.suggest_int('max_depth', 10, 20)
        classifier = BalancedRandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=47,
            class_weight=class_weight,
            max_features=max_features
        )




    # Fitting the classifier to the training data
    classifier.fit(X_final, y_train_resampled)

    # Predicting the probabilities for the test set
    y_pred_proba = classifier.predict_proba(X_final_test)[:, 1]

    # Calculating and returning the ROC AUC score for the predictions
    return roc_auc_score(y_test, y_pred_proba)


In [28]:
X_test_transformed = preprocessor.transform(X_test)

In [29]:
# Get the selected feature indices from rfecv.support_
selected_feature_indices = np.where(rfecv.support_)[0]

# Select the features from X_test_transformed using the selected feature indices
X_test_selected = X_test_transformed[:, selected_feature_indices]

# Select corresponding feature names
selected_feature_names = all_feature_names[selected_feature_indices]

# Create DataFrame with selected features and names
X_final_test = pd.DataFrame(X_test_selected, columns=selected_feature_names)

In [30]:
# Optuna study
study = optuna.create_study(direction='maximize', sampler=TPESampler())
study.optimize(objective, n_trials=50)


[I 2025-01-20 04:24:56,436] A new study created in memory with name: no-name-01020e64-182c-4cce-ba2d-14e12edcb038
[I 2025-01-20 04:29:45,579] Trial 0 finished with value: 0.7075592501626886 and parameters: {'classifier': 'SVC', 'C': 0.7916687767170469, 'kernel': 'linear'}. Best is trial 0 with value: 0.7075592501626886.
[I 2025-01-20 04:29:48,072] Trial 1 finished with value: 0.6960019406017265 and parameters: {'classifier': 'RandomForest', 'n_estimators': 240, 'max_depth': 3}. Best is trial 0 with value: 0.7075592501626886.
[I 2025-01-20 04:29:49,266] Trial 2 finished with value: 0.6859140770596152 and parameters: {'classifier': 'RandomForest', 'n_estimators': 202, 'max_depth': 1}. Best is trial 0 with value: 0.7075592501626886.
[I 2025-01-20 04:31:11,403] Trial 3 finished with value: 0.7252765707328337 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 239, 'learning_rate': 0.04010075978856494, 'max_depth': 11}. Best is trial 3 with value: 0.7252765707328337.
[I 2025-

In [31]:
# Best results
print(f"Best parameters: {study.best_params}")
print(f"Best ROC AUC score: {study.best_value}")


Best parameters: {'classifier': 'LGBMClassifier', 'n_estimators': 273, 'learning_rate': 0.10233531202936266, 'num_leaves': 13}
Best ROC AUC score: 0.7607415337751001


In [32]:

# Train the best model on the full training data
best_params = study.best_params
best_classifier_name = best_params.pop('classifier')

if best_classifier_name == 'RandomForest':
    best_model = RandomForestClassifier(**best_params, random_state=47)
elif best_classifier_name == 'LGBMClassifier':
    best_model = LGBMClassifier(**best_params, random_state=47)
elif best_classifier_name == 'GradientBoosting':
    best_model = GradientBoostingClassifier(**best_params, random_state=47)
elif best_classifier_name == 'SVC':
    best_model = SVC(**best_params, probability=True, random_state=47)
else:
    best_model = BalancedRandomForestClassifier(**best_params, random_state=47)


In [40]:
# Storing the best-performing model after training and evaluation
best_model

LGBMClassifier(learning_rate=0.10233531202936266, n_estimators=273,
               num_leaves=13, random_state=47)

In [156]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

# Fit the best model on the resampled training data
best_model.fit(X_final, y_train_resampled)

# Predict probabilities on the test set
y_prob = best_model.predict_proba(X_final_test)[:, 1]

# Define a threshold for classification
threshold = 0.082  # Default threshold, can be tuned for better performance
y_pred = (y_prob >= threshold).astype(int)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)  # Handle cases with no positive predictions
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)
confusion_mat = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print(confusion_mat)

Accuracy: 0.7377
Precision: 0.1356
Recall: 0.6076
F1 Score: 0.2217
ROC AUC Score: 0.7607
[[2700  918]
 [  93  144]]


In [133]:
thresholds = np.arange(0.01, 0.9, 0.001)
best_threshold = 0.5
best_f1 = 0.0
for t in thresholds:
    y_pred = (y_prob >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t
print(f"Best Threshold: {best_threshold}, Best F1 Score: {best_f1}")

Best Threshold: 0.10299999999999991, Best F1 Score: 0.24316109422492402
